# Estudo de Caso 4 - Geração de Rostos Humanas com Redes Generativas

Este Estudo de Caso demonstra na prática como construir um modelo generativo capaz de criar faces humanas realísticas usando Linguagem Python e PyTorch. O modelo é criado com arquitetura Deep Convolutional Generative Adversarial Networks (DCGANs), uma extensão das Generative Adversarial Networks (GANs), que usam camadas convolucionais em vez de apenas camadas completamente conectadas.

Fonte: http://mmlab.ie.cuhk.edu.hk/projects/CelebA.htmlAcompanhe


## 1. Instalando e carregando pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Instala o Pytorch
!pip install -q torch==2.0.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.0 requires torch==1.13.0, but you have torch 2.0.0 which is incompatible.


In [3]:
# Instala o Torchvision
!pip install -q torchvision==0.15.1

In [4]:
# Imports
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Estudo de Caso 4 - Redes Generativas" --iversions

Author: Estudo de Caso 4 - Redes Generativas

matplotlib : 3.5.2
torchvision: 0.15.1
numpy      : 1.21.5
torch      : 2.0.0



## 2. Definindo parâmetros globais

In [6]:
# Seed para reproduzir o mesmo resultado
manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [7]:
# Diretório raiz dos dados
dataroot = 'dados/celeba'

In [8]:
# Número de workers para o dataloader
workers = 2

In [9]:
# Número de GPUs disponíveis. Use 0 para o modo de CPU.
ngpu = 1

## 3. Preparando a pasta de imagens e o dataloader

In [10]:
# Define o tamanho padrão das imagens
image_size = 64

In [12]:
# Cria o dataset com a pasta de imagens
dataset = dset.ImageFolder(root = dataroot,
                           transform = transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size), # pega o miolo da imagem, se for maior que 64x64
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

FileNotFoundError: Couldn't find any class folder in dados/celeba.

A arquitetura GAN é aprendizado não supervisionado, ou seja, não temos dados de teste para avaliar o modelo. Não sabemos qual é a saída do modelo, mas esse é o objetivo do modelo. 

In [ ]:
# Batch size para o treinamento
batch_size = 128

In [ ]:
# Cria o dataloader
dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size = batch_size,
                                        shuffle = True,
                                        num_workers = workers)

In [ ]:
# Device usado no treinamento
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
# Visualiza o device
print(device)